<a href="https://colab.research.google.com/github/Absinthe31/ProgettoINFORET/blob/develop/Matches_predictor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install and import libraries

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install scrapy --quiet
!pip install snscrape --quiet
!pip install selenium --quiet
!pip install pyvirtualdisplay --quiet
#!pip install nltk --quiet
!pip install deep_translator --quiet
!pip install textblob --quiet
!pip install unidecode --quiet
!pip install Levenshtein --quiet


#!pip install -r requirements.txt

!python -m spacy download en_core_web_lg
!python -m spacy download en_core_web_sm
!python -m textblob.download_corpora

     |████████████████████████████████| 271 kB 36.6 MB/s 
     |████████████████████████████████| 93 kB 3.0 MB/s 
     |████████████████████████████████| 57 kB 5.6 MB/s 
     |████████████████████████████████| 3.1 MB 51.4 MB/s 
     |████████████████████████████████| 254 kB 69.0 MB/s 
     |████████████████████████████████| 4.1 MB 63.0 MB/s 
     |████████████████████████████████| 74 kB 4.0 MB/s 
     |████████████████████████████████| 5.2 MB 31.2 MB/s 
     |████████████████████████████████| 140 kB 76.3 MB/s 
     |████████████████████████████████| 384 kB 73.6 MB/s 
     |████████████████████████████████| 58 kB 6.7 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.23.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you have urllib3 1.26.12 which is incompatible.
     |████████████████████████████████| 128 kB 47.9 MB/s 
     |█████████████

In [3]:
!sudo apt-get -qq update
!sudo apt-get -qq -y install xvfb

debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package xvfb.
(Reading database ... 123942 files and directories currently installed.)
Preparing to unpack .../xvfb_2%3a1.19.6-1ubuntu4.11_amd64.deb ...
Unpacking xvfb (2:1.19.6-1ubuntu4.11) ...
Setting up xvfb (2:1.19.6-1ubuntu4.11) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...


In [4]:
cd /content/drive/MyDrive/Github/ProgettoINFORET/skysport

/content/drive/MyDrive/Github/ProgettoINFORET/skysport


In [ ]:
#!sudo dpkg -i google-chrome-stable_current_amd64.deb
#!sudo apt -qq -y --fix-broken install
#!sudo dpkg -i google-chrome-stable_current_amd64.deb

In [5]:
import selenium
import snscrape.modules.twitter as sntwitter
import datetime
import numpy as np
import pandas as pd
import requests
import spacy
import json
import copy
import os
import time
import re
import nltk
import urllib.request
import Levenshtein as lev

from nltk.corpus import sentiwordnet as swn
from nltk.corpus import wordnet as wn
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from pyvirtualdisplay import Display
from selenium.common.exceptions import NoSuchElementException, StaleElementReferenceException, ElementClickInterceptedException
from nltk.corpus import stopwords
from string import punctuation, digits
from deep_translator import GoogleTranslator
from tqdm.notebook import tqdm
from textblob import TextBlob
from textblob.taggers import NLTKTagger

from nltk.corpus import wordnet as wn
from nltk.stem import WordNetLemmatizer


nlp = spacy.load("en_core_web_sm")


        
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('sentiwordnet')
nltk.download('omw-1.4')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
#nltk.download('movie_reviews')
translator = GoogleTranslator(source='it', target='en')
#os.chmod("./chromedriver", 0o777)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package sentiwordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/sentiwordnet.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [6]:
!apt -qq install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  chromium-browser chromium-browser-l10n chromium-codecs-ffmpeg-extra
Suggested packages:
  webaccounts-chromium-extension unity-chromium-extension
The following NEW packages will be installed:
  chromium-browser chromium-browser-l10n chromium-chromedriver
  chromium-codecs-ffmpeg-extra
0 upgraded, 4 newly installed, 0 to remove and 28 not upgraded.
Need to get 91.7 MB of archives.
After this operation, 309 MB of additional disk space will be used.
Selecting previously unselected package chromium-codecs-ffmpeg-extra.
(Reading database ... 123949 files and directories currently installed.)
Preparing to unpack .../chromium-codecs-ffmpeg-extra_105.0.5195.102-0ubuntu0.18.04.1_amd64.deb ...
Unpacking chromium-codecs-ffmpeg-extra (105.0.5195.102-0ubuntu0.18.04.1) ...
Selecting previously unselected package 

# Data processing

## Articles scraping

In [ ]:
!rm scrapedUrls.json
!scrapy crawl titles -o scrapedUrls.json

## Scraping teams and players stats

In [7]:


dataframe = pd.read_json(path_or_buf='./scrapedUrls.json')

token = '9592c1c916514c54a712c48fe03a14b4'

key = "8e5a90e4536f74e302656ba3f717456413faa66a1804d165187a7b398ca66086"

uri2 = f"https://apiv3.apifootball.com/?action=get_events&from=2020-08-10&to=2021-07-10&league_id=207&APIkey={key}"

matches = requests.get(uri2)
    

uri = 'https://api.football-data.org/v4/competitions/SA/teams?season=2020'
headers = { 'X-Auth-Token': token}

teams = requests.get(uri, headers=headers)

xteams = copy.deepcopy(teams)
    

teams = [team['shortName'] for team in teams.json()['teams']]

teams_dict = { i : pd.DataFrame() for i in teams }


def adjust_name(name):

    return name.split(' ')[0].lower()

for team in teams:
    
    teams_dict[team] = dataframe[[(adjust_name(team) in x) for x in dataframe.tags]]

In [8]:

def remove_prefix(string):
    return string.replace("AC ", "").replace(" Calcio", '').replace("FC ", "").replace("Hellas ","").replace("nazionale", "")


def remove_adds(string):

    if '+' in string:

        return string.split("+")[0]

    elif '-' in string:

        return string.split("-")[0]

    else:

        return string

    
def get_numbers(string): 

    listx =  ' '.join([remove_adds(n) for n in string.split(' ')])
    sentence = listx
    #print('LIST')
    #print(listx,'\n')
    numbers = [float(s) for s in re.findall(r'-?\d+\.?\d*', sentence)]
    #print('NUMBERS')
    #print(numbers)
    numbers = [(int(n) if n.is_integer() else n) for n in numbers]
    #print(numbers)
    try:
        if(len(numbers) == 9) or (len(numbers) == 11):
            numbers[2] /= 1000
        else:
            numbers[3] /= 1000
    except IndexError:
        pass
    numbers = [n for n in numbers if n<1000]

    return numbers


def get_team_player(string, leaderboard):
    name = []
    for team in leaderboard.Team:
        if team in string:
            string = string.replace(team,'')
            name.append(team)
            #break

    return (string,name)

def get_table(i,l,leaderboard,driver):
    try:
      pagination = driver.find_elements('xpath',"//ul//li[contains(@class,'page')]")
      pagination[i].click()


    except NoSuchElementException:
      print("No such element get_table")
      return True

    #except ElementClickInterceptedException:
    #  print("Element click intercepted get_table")
    #  pass


    j = 1


    while True:
        try:
            b = driver.find_element('xpath',f"//div[@id='league-players']/table/tbody/tr[{j}]").text

            (c, team) = get_team_player(b,leaderboard)
            players = [str for str in c.split(' ') if (str.isalpha())]
            
            c = get_numbers(b)
            c.insert(1, ' '.join(players))
            c.insert(2, team)
        
            l.append(c) 
            #print(c)
            j+=1
            
            #time.sleep(2)
        except NoSuchElementException:
            break

In [9]:
#display = Display(size=(1920, 1080))  
#display.start()
url = 'https://understat.com/league/Serie_A/2020'

import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

options = webdriver.ChromeOptions()
options.add_argument('--disable-dev-shm-usage')    
options.add_argument('--headless')
options.add_argument('--no-sandbox')      
#options.binary_location = "/usr/bin/google-chrome-stable"
#chrome_driver_binary = "./chromedriver"
#d = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()),options = options)
d = webdriver.Chrome('chromedriver',options=options)
d.get(url)
def get_tables(s_date, e_date,driver):
    actions = ActionChains(driver) 
    driver = webdriver.Chrome('chromedriver',options = options)
    driver.set_window_size(1920, 1080)
    driver.get(url)

    driver.find_element('name','date-start').click()


    sDate  = driver.find_element('name','date-start') # set start date input element into variable
    sDate.clear() #clear existing entry
    sDate.send_keys(s_date) #add custom entry


    driver.find_element('name','date-end').click()

    eDate  = driver.find_element('name','date-end') # set start date input element into variable
    eDate.clear() #clear existing entry

    eDate.send_keys(e_date) #add custom entry
    eDate.send_keys(Keys.TAB)
    #actions = actions.send_keys(Keys.TAB)
    #actions.perform()

    #element = driver.find_element('xpath',"//body")
    #element.send_keys(Keys.TAB)


    #SEMBRA FUNZIONARE  --  NON è VERO, ORA DA TIMEOUT EXCEPTION, PROVO A FIXARE -- NON SO, MI SA CHE L'ECCEZIONE è CAUSATA DAL FATTO CHE ANCHE ASPETTANDO 10 SECONDI
    #COMUNQUE L'ELEMENTO NON C'è, PROVO CON 15 SECONDI MA BOH, NON SO -- BOH, NON VA NEMMENO COSì, SEMBRA CHE PROPRIO NON VIENE CARICATO L'ELEMENTO

    try:

      p = WebDriverWait(driver, 15).until( EC.visibility_of_element_located((By.XPATH, "//div[@id='league-chemp']/table/thead/tr"))).text


    except TimeoutException:
      print('ERRORE')
      #Non saprei come gestire se non c'è l'elemento dopo 15 secondi, perchè anche ri-eseguendo può essere che non c'è ancora
      pass


    playersHeader = driver.find_element('xpath',"//div[@id='league-players']/table/thead/tr").text


    i = 1
    l = [] 

    while True:
        try:
            b = driver.find_element('xpath',f"//div[@id='league-chemp']/table/tbody/tr[{i}]").text
            
            teams = [str for str in b.split(' ') if (str.isalpha())]
            #print(teams)
            #not "abc" in s
            
            c = get_numbers(b)
            c.insert(1, ' '.join(teams))
            
            l.append(c) 

            #print(l)
            #leaderboard.loc[i] = b.split(' ')
            i+=1
            
        except NoSuchElementException:
            break

    leaderboard = pd.DataFrame(l, columns=p.split(' '))
    leaderboard = leaderboard.set_index('№')
    #print('CLASSIFICA')
    #print(leaderboard)

    driver.find_element('name','players-date-start').click()


    sDate  = driver.find_element('name','players-date-start') # set start date input element into variable
    sDate.clear() #clear existing entry
    sDate.send_keys(s_date) #add custom entry


    driver.find_element('name','players-date-end').click()

    eDate  = driver.find_element('name','players-date-end') # set start date input element into variable
    eDate.clear() #clear existing entry
    eDate.send_keys(e_date) #add custom entry

    driver.find_element('id',"players-filter").click()


    time.sleep(1.5)

    try:
      #WebDriverWait(driver, 15).until( EC.element_to_be_clickable((By.XPATH, "//ul//li[contains(@class,'page')]")))
      pagination = driver.find_elements('xpath',"//ul//li[contains(@class,'page')]")
    except TimeoutException:
      print('ERRORE 2')
      #Non saprei come gestire se non c'è l'elemento dopo 15 secondi, perchè anche ri-eseguendo può essere che non c'è ancora
      pass


    #pagination = driver.find_elements('xpath',"//ul//li[contains(@class,'page')]")

    #print('PAGINATION:')
    #print(pagination[-1].text)
    max_page = int(pagination[-1].text)-2
    j = int(pagination[3].text)
    

    l = []

    for i in range(0,4):
        if get_table(i,l,leaderboard,driver):
            break




    while True:
        
        if get_table(-2,l,leaderboard,driver) or (j == max_page):
            break
        j += 1



    get_table(-1,l,leaderboard,driver)

    #print(l)

    players_stats = pd.DataFrame(l, columns=playersHeader.split(' '))
    players_stats = players_stats.set_index('№')
    players_stats = players_stats.dropna()
    players_stats['Min'] = players_stats['Min'] * 1000
    leaderboard['W'] = leaderboard['W'] * 1000

    #input('Premi INVIO per chiudere l\'istanza di Chrome lanciata')
    driver.quit()

    return leaderboard,players_stats

In [10]:
l,p = get_tables('', '',d)

In [ ]:
l

,Team,M,W,D,L,G,GA,PTS,xG,xGA,xPTS
№,,,,,,,,,,,
1,Inter,38,28.0,7,3,89,35,91,78.35,40.59,77.32
2,AC Milan,38,24.0,7,7,74,41,79,75.05,48.84,68.75
3,Atalanta,38,23.0,9,6,90,47,78,82.01,39.94,77.57
4,Juventus,38,23.0,9,6,77,38,78,83.53,38.33,79.47
5,Napoli,38,24.0,5,9,86,41,77,74.98,41.88,73.83
6,Lazio,38,21.0,5,12,61,55,68,58.70,48.84,57.38
7,Roma,38,18.0,9,11,68,55,63,72.11,50.33,65.20
8,Sassuolo,38,17.0,11,10,64,56,62,61.61,54.82,57.31
9,Sampdoria,38,15.0,7,16,52,54,52,46.70,58.63,44.07


In [11]:
def get_lineup(lineup):
  players_list = list()
  for player in lineup:
    players_list.append(player['lineup_player'].lower())

  return players_list

In [12]:
matches_df = pd.json_normalize(matches.json())

matches_df.match_date = matches_df.match_date + 'T' + matches_df.match_time + ':00Z'
matches_df['lineup.home.starting_lineups'] = matches_df['lineup.home.starting_lineups'].apply(get_lineup)
matches_df['lineup.away.starting_lineups'] = matches_df['lineup.away.starting_lineups'].apply(get_lineup)

matches_df = matches_df.rename(columns={'match_date':'date','match_round':'day','match_hometeam_name':'hteam_name','match_awayteam_name':'ateam_name','match_hometeam_ft_score':'hteam_score','match_awayteam_ft_score':'ateam_score','lineup.home.starting_lineups':'hteam_lineup','lineup.away.starting_lineups':'ateam_lineup'})
matches_df.insert(7, 'winner', None)
matches_df.hteam_name = matches_df.hteam_name.apply(remove_prefix)
matches_df.ateam_name = matches_df.ateam_name.apply(remove_prefix)
matches_df.hteam_name = matches_df.hteam_name.apply(str.lower)
matches_df.ateam_name = matches_df.ateam_name.apply(str.lower)

matches_df = matches_df[['match_id','date','day','hteam_name','ateam_name','hteam_score','ateam_score','winner','hteam_lineup','ateam_lineup','statistics']]

## Scraping tweets

In [ ]:

def create_string(team):

    all_teams = list(teams)

    other_teams = [f'({t})' for t in all_teams if (team not in t)]

    query_str_tmp = f'({team} #serieA)' + ' -(calciomercato)' + ' -' + ' -'.join(other_teams)

    #query_str =  f'({hteam} {ateam})' + ' -' + ' -'.join(other_teams)
    
    return query_str_tmp


'''def create_string(hteam, ateam):

    all_teams = list(teams)

    other_teams = [f'({team})' for team in all_teams if (hteam not in team) and (ateam not in team)]

    query_str_tmp = f'(#{}'

    query_str =  f'({hteam} {ateam})' + ' -' + ' -'.join(other_teams)
    
    return query_str'''


def create_date_range(date,range):
    
    date = date.split('T')[0]
    d = datetime.datetime.strptime(date, "%Y-%m-%d")
    delta = datetime.timedelta(days=-range)
    date_from = d + delta
    query_str = f' since:{date_from.date()} until:{date}'
    return query_str


def get_tweets(matchday,tweets_df):

    tweets = [None for x in range(10)]
    i = 0
    matches_day = matches_df[matches_df.day == matchday].values
    for match in matches_day:
        h_team = match[2]
        a_team = match[3]
        date = match[0]
        df = get_match_tweets(matchday,h_team,a_team,date)
        tweets[i] = df #per ogni partita salvo i tweet relativi ai team che vi partecipano
        #a_series = pd.Series(df, index = tweets_df.columns)
        #tweets_df = tweets_df.append(a_series, ignore_index=True)
        tweets_df = pd.concat([tweets_df,df],axis=0)
        i+=1
    print(i)
    return tweets_df

In [ ]:


def get_match_tweets(matchday,hteam,ateam,date):
    

    # Creating list to append tweet data 
    tweets_list1 = []

    #print(create_string(hteam) + create_date_range(date,21) + ' lang:it min_faves:0 -filter:links filter:verified')
    i = 0
    # Using TwitterSearchScraper to scrape data and append tweets to list
    for i,tweet in enumerate(sntwitter.TwitterSearchScraper(create_string(hteam) + create_date_range(date,21) + ' lang:it min_faves:0 -filter:links').get_items()): #declare a username 
        if i>1000: #number of tweets you want to scrape
            break
        tweets_list1.append([tweet.date,hteam,matchday, tweet.id, tweet.content, tweet.user.username, tweet.likeCount]) #declare the attributes to be returned
        
    #print(create_string(ateam) + create_date_range(date,21) + ' lang:it min_faves:0 -filter:links filter:verified')    
    for i,tweet in enumerate(sntwitter.TwitterSearchScraper(create_string(ateam) + create_date_range(date,21) + ' lang:it min_faves:0 -filter:links').get_items()): #declare a username 
        if i>1000: #number of tweets you want to scrape
            break
        tweets_list1.append([tweet.date,ateam,matchday, tweet.id, tweet.content, tweet.user.username, tweet.likeCount]) #declare the attributes to be returned

    if i > 0:
        # Creating a dataframe from the tweets list above 
        tweets_df1 = pd.DataFrame(tweets_list1, columns=['Datetime','Team','Matchday', 'Tweet_Id', 'Text', 'Username', 'Likes'])

        start_time = datetime.datetime.strptime(date, "%Y-%m-%dT%H:%M:%SZ")
        
        tweets_df1 = tweets_df1[[( (datetime.datetime.strptime(str(datex), "%Y-%m-%d %H:%M:%S+00:00").day == start_time.day) and (datetime.datetime.strptime(str(datex), "%Y-%m-%d %H:%M:%S+00:00").hour < start_time.hour) or not(datetime.datetime.strptime(str(datex), "%Y-%m-%d %H:%M:%S+00:00").day == start_time.day)) for datex in tweets_df1.Datetime ]]
        #print(tweets_list1)
        return tweets_df1
    else:
        return None

In [ ]:
df_tweets = pd.DataFrame(columns=['Datetime','Team','Matchday', 'Tweet_Id', 'Text', 'Username', 'Likes'])
for i in range(39):
    df = get_tweets(i,df_tweets)
    temp = pd.concat([temp,df],axis=0)
temp.to_csv(r'tweets.csv', index = False)

In [ ]:
df_tweets = pd.read_csv('tweets.csv')

In [ ]:
tokenize = lambda text: [x.lemma_.lower() for x in text if x.pos_ in ['NOUN', 'PROPN', 'VERB', 'ADJ', 'ADV']]

# Data cleaning 

In [13]:
from nltk.corpus import stopwords
stop_words = stopwords.words('italian')
stop_words.extend(['sky', 'sport', 'serie', 'campionato', 'giornata', 'classifica','foto', '|'])

In [14]:
def remove_words(string, list):

    for word in list:
        if word in string:
            string = string.replace(word, '')

    return string

def remove_emojs(text):
    emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002500-\U00002BEF"  # chinese char
        u"\U00002702-\U000027B0"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642" 
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  # dingbats
        u"\u3030"
                           "]+", flags=re.UNICODE)
    text = emoji_pattern.sub(r'', text) # no emoji
    return text

In [15]:
new_df = dataframe.title.str.lower()
new_df = copy.deepcopy(dataframe)
new_df['title'] = dataframe['title'].str.lower()
new_df['text'] = dataframe['text'].str.lower()

word_rm_list = [' sky', 'sport', 'foto', '|', 'video', 'news', '"', "'"]

new_df['title'] = new_df['title'].apply(remove_words, list = word_rm_list)
new_df['text'] = new_df['text'].apply(remove_words, list = word_rm_list)

new_df = new_df.drop(new_df[[(('serie a donne' in x) or ('calcio femminile' in x) or not('serie a' in x) or ('fotogallery' in x)) for x in new_df.tags]].index)

new_df['text'] = new_df['title'] + new_df['text'] 
new_df['text'] = new_df['text'].str.replace('\xa0', '', regex=True)
new_df = new_df.drop(['title'], axis=1)

In [ ]:
word_rm_list_tweets = ['#SerieA', '#', '@', '#SerieATIM' , '#seriea','seriea' ]

df_tweets['Text'] = df_tweets['Text'].str.lower()
df_tweets['Text'] = df_tweets['Text'].apply(remove_words, list = word_rm_list_tweets)
df_tweets['Text'] = df_tweets['Text'].apply(remove_emojs)

In [16]:
players_team = pd.DataFrame()
players_team['Player'] = p['Player'].str.lower()
players_team['Team'] = p['Team'].apply(lambda x: [y.lower() for y in x])

In [17]:
def removearticles(text):
    text = text.split(" ")
    if len(text) > 1:
        return " ".join(text[1:]).lower()
    else:
        return text[0].lower()
    

team_names = pd.read_csv('./soprannomi.csv')
team_names['soprannome'] = team_names['soprannome'].apply(removearticles)
team_names['squadra'] = team_names['squadra'].apply(str.lower)


team_names.loc[56,'squadra'] = 'napoli'
team_names.loc[56,'soprannome'] = 'naples'
team_names.loc[57,'squadra'] = 'roma'
team_names.loc[57,'soprannome'] = 'rome'
team_names.loc[58,'squadra'] = 'torino'
team_names.loc[58,'soprannome'] = 'turin'
team_names.loc[59,'squadra'] = 'benevento'
team_names.loc[59,'soprannome'] = 'giallorossi'
team_names.loc[58,'squadra'] = 'spezia'
team_names.loc[58,'soprannome'] = 'spice'

subjects_func = lambda text : [tok for tok in text if (tok.dep_ == "nsubj")]

In [18]:
def subtrees(node):
    if not node.children:
        return []

    result = [ list(node.children) ]
    for child in node.children:
        result.extend(subtrees(child))

    return result

In [19]:
#NON FUNZIONA, CAPIRE COME FARE

tokenizer = nltk.data.load('tokenizers/punkt/italian.pickle')
sentences_list = []
sentences_ita = pd.DataFrame(columns=["date","text"])
prova = "sulla suggestione messi: tutti lo desidererebbero ma resterà al barcellona. si tratta di fantacalcio  atalanta-inter: cosi la partita nel post partita tra atalanta e inter, l’attacco di  antonio conte . prima e dopo la sfida che ha dato il secondo posto ai nerazzurri, le parole dell’amministratore delegato  beppe marotta . intanto sull’episodio che ha visto coinvolto marcelo brozovic in settimana"
sentences_list = tokenizer.tokenize(prova)
#print(sentences_list)
i = 0 
for row in tqdm(new_df.values):
  date, tags, doc = row
  s = re.sub('\t+', '', doc)
  pattern = '\n' + '{2,}'
  s = re.sub(pattern, '.', s).strip()
  #print(row)
  sentences_list = tokenizer.tokenize(s)
  #sentences = tokenizer.tokenize(doc)
  for sentence in sentences_list:
    if 'probabile formazione:' in sentence or 'probabile formazione :' in sentence:
      sentences_ita.at[i,"date"] = date
      sentences_ita.at[i,"text"] = sentence
      i+=1

print(i)


  0%|          | 0/3174 [00:00<?, ?it/s]

354


In [ ]:
lemmatizer = WordNetLemmatizer()
scores = pd.DataFrame(columns=['date','subject','conjs','polarity','subjectivity','sentence','tags','team'])

def find_pos_split(positions, length):
    new_positions = [0]
    pos_i = 4999
    low = 0
    i = 0
    start_interval = 0

    while (length - start_interval) > 4999: 
        y = 1
        x = 0
        high = len(positions) - 1
        
        while low <= high: #42 <= 56 -> 42 <= 48 -> 46 <= 48 -> 48 <= 48
            mid = (high + low) // 2  #mid = 26 -> mid = 41 -> mid = 49 -> mid = 45 -> mid = 47 -> mid = 48

            if positions[mid] < pos_i: #2650 < 4999 -> 4298 < 4999 -> 4736 < 4999 -> 4894 < 4999
                low = mid+1 #low = 27 -> 42 -> 46 -> 48

            elif positions[mid] > pos_i: #5055 > 4999
                high = mid-1 #high = 48

            else:
                #caso in cui trovo esattamente 4999
                new_positions.append(positions[mid])
                break

        #siamo per forza in uno di questi due casi perchè 4999 non c'è nella lista
        if positions[mid] > pos_i:          
            while (positions[mid-y]-new_positions[i]) > 4999:
                y+=1
            mid = mid-y
            new_positions.append(positions[mid])
        elif positions[mid] < pos_i:
            while (positions[mid-x]-new_positions[i])  > 4999:
                x+=1
            mid = mid-x
            new_positions.append(positions[mid])
        
        pos_i = pos_i*2 # pos_i = 9998
        low = mid # low = 48
        start_interval = positions[mid]
        i += 1

    return new_positions[1:]

def getPolarity(text):
    blob = TextBlob(text)
    return blob.sentiment.polarity

def getSubjectivity(text):
   return TextBlob(text).sentiment.subjectivity

def penn_to_wn(tag):
    """
    Convert between the PennTreebank tags to simple Wordnet tags
    """
    if tag.startswith('J'):
        return wn.ADJ
    elif tag.startswith('N'):
        return wn.NOUN
    elif tag.startswith('R'):
        return wn.ADV
    elif tag.startswith('V'):
        return wn.VERB
    
    return None

index = 0


tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
nltk_tagger = NLTKTagger()
translator = GoogleTranslator(source='it', target='en')
i = -1
for row in new_df.values:
    i = i+1
    
    subjects = []
    date, tags, doc = row
    print(date)
    pol_sum = 0
    subj_sum = 0
    subjects = None
    

    #tradurre doc
    if len(doc) < 5000:
        doc = translator.translate(doc)
    else:
        positions = [pos for pos, char in enumerate(doc) if (char == '.' or char == '?' or char == '!')]
       
        split_positions = find_pos_split(positions, len(doc))

        n_splits = len(split_positions)

        start = 0

        new_doc = ""
        new_doc_prova = ""

        for split in range(n_splits):

            new_doc_prova+=doc[start:split_positions[split]]
            new_doc += (translator.translate(doc[start:split_positions[split]]))
            start = split_positions[split]+1

        new_doc += (translator.translate(doc[start:len(doc)-1]))
        new_doc_prova += doc[start:len(doc)-1]

        doc = new_doc

    sentences = tokenizer.tokenize(doc)

    #dividere in frasi
    for sentence in sentences:
        
        try:
            blob = TextBlob(sentence,pos_tagger=nltk_tagger)
            
        except(TypeError):
            print(sentence)
            continue

        new_sentence = []
        for token, pos in blob.pos_tags:
            if penn_to_wn(pos) not in (wn.NOUN, wn.ADJ, wn.ADV, wn.VERB):
                continue
            new_sentence.append(lemmatizer.lemmatize(token,penn_to_wn(pos)))
    
        new_sentence = " ".join(new_sentence)
        doc=nlp(sentence)

        roots = [token for token in doc if token.pos_ == 'VERB']

        tokens = [{'text': token.text,'dep': token.root.dep_,'pos': token.root.pos_} for token in doc.noun_chunks if token.root.pos_ in ['NOUN', 'PROPN', 'PRON']]
        conjunctions = []
        
        #lista di soggetti e lista di liste congiunzioni

        subjects = []
        if len(roots) > 0:
            for r in roots:
                tree = list(r.lefts)
                for node in tree:
                    list_of_lists = subtrees(node)
                    list_of_lists.extend([list([node])])
                    subjects += [item for sublist in list_of_lists for item in sublist if item.pos_ in ['NOUN', 'PROPN']]
        else:
            subjects = []

         


        for token in doc:
            if token.conjuncts:
                conjuncts = token.conjuncts             # tuple of conjuncts
                tuples = [token.text]
                for conj in conjuncts:
                    span = doc[conj.i: conj.i+1]        # Here's span
                    tuples.append(span.text)
                if token.text not in conjunctions and (token.pos_ in ['NOUN', 'PROPN'] and conj.pos_ in  ['NOUN', 'PROPN']):
                    conjunctions.extend([tuples])

        conj_subjects = []
        for conj in conjunctions:
            if set(conj) <= set(subjects) and set(conj) <= set([x.lower() for x in teams]): #inefficente calcolare teams
                conj_subjects.extend(conj)

        
        pol = getPolarity(new_sentence)
        subj = getSubjectivity(new_sentence)
    
        scores.loc[index,'date'] = date
        scores['subject'].astype('object')
        scores.loc[index,'subject'] = subjects
        scores.loc[index, 'conjs'] = conj_subjects 
        scores.loc[index,'polarity'] = pol
        scores.loc[index,'subjectivity'] = subj
        scores.loc[index,'sentence'] = sentence
        scores.loc[index,'tags'] = tags

        index += 1


scores.to_csv("./scores1.csv")

In [20]:
def list_to_tuple(listx):
    if type(listx) == list:
        return tuple(listx)
    return listx

def list_to_string(listx):
    if type(listx) == list:
        return listx[0]
    return listx


In [21]:
scores = pd.read_csv('scores1.csv', lineterminator='\n', index_col=0)
scores["team"] = scores["team"].apply(list_to_string)
#scores[['team','polarity']].groupby("team").sum().reset_index().sort_values("polarity",ascending=False)

In [ ]:
from unidecode import unidecode

personal_pron = [ "i", "he", "it", "we", "they"]
#Assegnare ad una squadra il punteggio di una frase SE:
#   -La frase contiene il nome della squadra
#   -La frase contiene un sinonimi delle squadre
#   -La frase contiene  nomi di giocatori
#   -La frase contiene pronomi personali. In questo caso assegnare il punteggio all'ultima squadra alla quale è stato assegnato un punteggio,
#   NELLA FRASE PRECEDENTE

#Non assegnare punteggio a nessuna squadra SE
#   -Altrimenti

scores['team'].astype('object')

previous_assignment = ""
ever_assigned = False
assigned_team = ""


#modifica players_team e teams
def normalize_team_name(team):

    team = team.replace(" Calcio", "")
    team = team.replace(" calcio", "")
    team = team.replace("AC ", "")
    team = team.replace("ac ", "")

    return team

teams = [normalize_team_name(x) for x in teams]


for index,row in enumerate(tqdm(players_team.values)):
    #print(type(players_team.iloc[index]['Team']))
    new_list = []
    if isinstance(players_team.iloc[index]['Team'], list):

      for team in players_team.iloc[index]['Team']:
          #print(team)
          #print( players_team.iloc[index]['Team'])
          #print(team, type(team)) #team è una lista in teoria perchè un giocatore puo avere piu team
          new_list.extend([normalize_team_name(team)])
          print(new_list)
          #['juventus','milan']
     # print("new list:")
     # print(new_list)
      players_team['Team'] = players_team['Team'].astype(object)
      players_team.at[index,'Team'] = new_list
    else:
      players_team.at[index,'Team'] = players_team.iloc[index]['Team']

to_be_added = []


for index in tqdm(range(len(scores.index))):
    #print(index)
    assigned = False


    conjs = scores.loc[index,'conjs']
    for team in conjs:
        tmp = list(scores.loc[index])
        tmp[6] = team
        to_be_added.extend(list(scores.loc[index]))


    if len(conjs) == 0: 
        for subject in scores.loc[index,'subject']:
            #conjunctions contiene i team che fanno parte di congiunzioni
            assigned_now = False

            for team in [x.lower() for x in teams]:

                if subject.text.lower() == team:
                    print("Assigned 1, by team",team)
                    assigned_team = team
                    assigned = True
                    ever_assigned = True
                    assigned_now = True
                    break

            if assigned_now:
                previous_assignment = assigned_team
                break

            for soprannome in team_names.values:

                if len(team_names[[soprannome[1] == y for y in team_names.soprannome]]) > 1:
                    break
                if subject.text.lower() == soprannome[1]:
                    print("Assigned 2, by soprannome",soprannome[0])
                    assigned_team = soprannome[0]
                    assigned = True
                    ever_assigned = True
                    assigned_now = True

            if assigned_now:
                previous_assignment = assigned_team
                break

            for player in players_team.values:
                
                try:
                    x = player[0].split(" ")
                    x = x[1]
                except IndexError:
                    x = player[0]
                    
                if unidecode(subject.text.lower()) == unidecode(x):   #SE 'I' è CONTENUTA NEL NOME DEL GIOCATORE

                    #GESTIRE
                    tmp_df = players_team[[unidecode(x) in y for y in players_team.Player]]
                    
                    for tmp_index in tmp_df.index:
                        try:
                            tmp_df.loc[tmp_index,'Player'] = tmp_df.loc[tmp_index,'Player'].split(" ")[1]
                        except IndexError:
                            pass

                    if len(tmp_df[[unidecode(x) in y for y in tmp_df.Player]]) > 1:
                        break
                    
                    print("Assigned 3, by player name",subject.text.lower(), player[1])
                    if len(player[1]) > 1:
                        break
                    print(player[1][0])
                    assigned_team = player[1][0]
                    assigned = True
                    ever_assigned = True
                    assigned_now = True
        
            if assigned_now:
                previous_assignment = assigned_team
                break


            if not assigned:
            
                teams_set = set([x.lower() for x in teams])
                tags_set = set(scores.loc[index,'tags'])
                words_set = set(nlp(scores.loc[index,'sentence']))

                teams_tags = teams_set.intersection(tags_set)
                teams_tags_words = teams_tags.intersection(words_set)


                if len(teams_tags) == 1:
                    assigned_team = teams_tags.pop() #caso in cui c'è un solo team nei tag
                    assigned = True
                    ever_assigned = True
                    print("Assigned 5, by tag", previous_assignment)

                elif (len(teams_tags) > 1) and (len(teams_tags_words) == 1):
                    assigned_team = teams_tags_words.pop() #caso in cui ci sono più team nei tag ma c'è un solo team nella frase
                    assigned = True
                    ever_assigned = True
                    print("Assigned 5, by tag and sentence", previous_assignment)

            if  not assigned and ever_assigned:
                assigned_team = previous_assignment
                assigned = True
                print("Assigned 4, by previous", previous_assignment)

            if assigned:
                previous_assignment = assigned_team


        if len(scores.loc[index,'subject']) == 0:
            assigned_team = previous_assignment

        scores.at[index,'team'] = assigned_team

    
scores.append(pd.DataFrame(to_be_added))


scores.to_csv("./scores2.csv")

In [ ]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
stop_words = set(stopwords.words('english'))

block = ""
block_list = []
for index in tqdm(df_tweets.index):
    #creiamo blocchi <=4999
    text = df_tweets.loc[index,'Text']
    if len(block) + len(text) <4998:
        block += df_tweets.loc[index,'Text'] + '#'
        #print(block)
    else:
        block_list.extend([block])
        if len(block) > 4999:
            print(len(block))
        block = ""
        index -= 1 

#qualcosa non va perchè vengono messe le lettere al posto dei blocchi boh

print("Start translating")
for i,block in enumerate(tqdm(block_list)):
    #print(len(block_list[i]))
    block_list[i] = translator.translate(block)
    if i == 100:
        break

#salvare in un csv


#block_list[0] = dsfdsfdsfdsfdsdsffffdsf#fdsfdsfdsfsdfdfsd#dsfsdfdfsdfsdfsdsfsd
tweets_df_en = copy.deepcopy(df_tweets)
index = 0
for block in tqdm(block_list):
    tweets_split = block.split('#')
    for t in tweets_split:
        word_tokens = word_tokenize(t)
        t = [w for w in word_tokens if not w.lower() in stop_words]
        tweets_df_en.at[index,'Text'] = " ".join(t)  #ho modificato qui perchè prima mettevi in Text una lista di parole, ma sotto non adava bene
        index += 1

  0%|          | 0/37678 [00:00<?, ?it/s]

Start translating


  0%|          | 0/1108 [00:00<?, ?it/s]

  0%|          | 0/1108 [00:00<?, ?it/s]

In [ ]:
for i,tweet in enumerate(tqdm(tweets_df_en.Text)):

    pol = getPolarity(tweet)
    subj = getSubjectivity(tweet)

    tweets_df_en.loc[i,'polarity'] = pol
    tweets_df_en.loc[i,'subjectivity'] = subj


# Features

In [22]:
matches_df['winner'] = ['H' if match[5] > match[6] else 'A' if match[5] < match[6] else 'D' for match in matches_df.values]

In [23]:
league_df = pd.DataFrame(columns=['match_id','date','time','matchday','team','opponent','place','result','s_goals', 'c_goals','team_streak','opp_streak','team_last_five','opp_last_five'])

i = 0
for row in matches_df.values:
    result_home_team = ('W' if row[5] > row[6] else 'L' if row[5] < row[6] else 'D')
    result_away_team = ('W' if row[6] > row[5] else 'L' if row[6] < row[5] else 'D')
    date = datetime.datetime.strptime(row[1], "%Y-%m-%dT%H:%M:%SZ")
    league_df.loc[i] = [row[0],date.date(),date.hour,row[2],row[3],row[4],'H',result_home_team,row[5],row[6],None,None,None,None]
    i+=1
    league_df.loc[i] = [row[0],date.date(),date.hour,row[2],row[4],row[3],'A',result_away_team,row[6],row[5],None,None,None,None]
    i+=1

In [24]:
league_df['match_id'] = league_df['match_id'].astype('int')
league_df['matchday'] = league_df['matchday'].astype('int')
#league_df['date'] = pd.to_datetime(league_df['date'])
league_df['time'] = league_df['time'].astype('int')
league_df['s_goals'] = league_df['s_goals'].astype('int')
league_df['c_goals'] = league_df['c_goals'].astype('int')
league_df['team'] = league_df['team'].apply(str.lower)
league_df['opponent'] = league_df['opponent'].apply(str.lower)

In [25]:
for i,row in enumerate(tqdm(league_df.values)):
    streak_team = 0
    streak_opp = 0 #guarda sopra che ho scritto un commento ieri da farti leggere. per la roba dei tweet
    last_5_team = []
    last_5_opp = []
    if row[3] > 1: 
        
        df_team = league_df[((league_df['date'] < row[1]) & (league_df['team'] == row[4]))] #previous games of team at home
        df_opp = league_df[((league_df['date'] < row[1]) & (league_df['team'] == row[5]))] #recupero le partire precedenti che si riferiscono all'opponent
        
        df_team_last_5 = df_team.tail(5)
        df_opp_last_5 = df_opp.tail(5)

        #matches_df['winner'] = ['H' if match[4] > match[5] else 'A' if match[4] < match[5] else 'D' for match in matches_df.values]
        last_5_team = [1 if match[7] == 'W' else -1 if match[7] == 'L' else 0 for match in df_team_last_5.values]
        last_5_opp = [1 if match[7] == 'W' else -1 if match[7] == 'L' else 0 for match in df_opp_last_5.values]
        #print(last_5_team)
        #df_home_matches_team
        #print(df_team)
        for y,match in enumerate(df_team.values):

            if df_team.iloc[y]['result'] == 'W':
                streak_team+=1
            elif df_team.iloc[y]['result'] == 'D':
                streak_team+=0.5
            else:
                streak_team = 0
            #print(df_team)
            
        for y,match in enumerate(df_opp.values):    
            if df_opp.iloc[y]['result'] == 'W':
                streak_opp+=1
            elif df_opp.iloc[y]['result'] == 'D':
                streak_opp+=0.5
            else:
                streak_opp = 0

    if len(last_5_team) < 5:
        last_5_team = [0 for x in range(5 - len(last_5_team))] + last_5_team
    
        last_5_opp = [0 for x in range(5 - len(last_5_opp))] + last_5_opp

    league_df.loc[i,'team_streak'] = streak_team
    league_df.loc[i,'opp_streak'] = streak_opp
    league_df['team_last_five'] = league_df['team_last_five'].astype('object')
    league_df['opp_last_five'] = league_df['opp_last_five'].astype('object')
    league_df.at[i,'team_last_five'] = last_5_team
    league_df.at[i,'opp_last_five'] = last_5_opp
    
    if i <= len(league_df)-2: 
        i+=1
        league_df.loc[i,'team_streak'] = streak_opp
        league_df.loc[i,'opp_streak'] = streak_team
        league_df.at[i,'team_last_five'] = last_5_opp
        league_df.at[i,'opp_last_five'] = last_5_team
        #league_df['rank']



  0%|          | 0/760 [00:00<?, ?it/s]

In [26]:
dates = [(datetime.datetime.strptime(day, "%Y-%m-%dT%H:%M:%SZ")).date() for day in list(matches_df['date'])]
dates = list(dict.fromkeys(dates))

In [27]:
def get_date_range(date,range):
    delta = datetime.timedelta(days=range)
    date_from = date - delta
    return date_from
def str_to_date(string):
    return datetime.datetime.strptime(string, "%d/%m/%Y").date()

In [ ]:
league_df.sort_values(['matchday','date'])

,match_id,date,time,matchday,team,opponent,place,result,s_goals,c_goals,team_streak,opp_streak,team_last_five,opp_last_five
0,59169,2020-09-19,18,1,fiorentina,torino,H,W,1,0,0,0,"[0, 0, 0, 0, 0]","[0, 0, 0, 0, 0]"
1,59169,2020-09-19,18,1,torino,fiorentina,A,L,0,1,0,0,"[0, 0, 0, 0, 0]","[0, 0, 0, 0, 0]"
2,59170,2020-09-19,20,1,verona,roma,H,W,3,0,0,0,"[0, 0, 0, 0, 0]","[0, 0, 0, 0, 0]"
3,59170,2020-09-19,20,1,roma,verona,A,L,0,3,0,0,"[0, 0, 0, 0, 0]","[0, 0, 0, 0, 0]"
4,59171,2020-09-20,12,1,parma,napoli,H,L,0,2,0,0,"[0, 0, 0, 0, 0]","[0, 0, 0, 0, 0]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
755,59518,2021-05-23,20,38,roma,spezia,A,D,2,2,1,1.5,"[-1, -1, 1, -1, 1]","[-1, 0, -1, 0, 1]"
756,59519,2021-05-23,15,38,inter,udinese,H,W,5,1,0,0,"[1, 1, 1, 1, -1]","[1, -1, 0, -1, -1]"
757,59519,2021-05-23,15,38,udinese,inter,A,L,1,5,0,0,"[1, -1, 0, -1, -1]","[1, 1, 1, 1, -1]"
758,59520,2021-05-23,20,38,napoli,verona,H,D,1,1,7.0,0.5,"[1, 0, 1, 1, 1]","[-1, 0, 0, -1, 0]"


In [29]:
info_dict = {}
for team in teams:
  team = remove_prefix(team).lower()
  info_dict[team] = 0

In [30]:
def reset_dict(dictx):
  for key in dictx.keys():
    dictx[key] = 0

In [31]:
from thinc.layers.list2array import list2array
formations_df = pd.DataFrame(columns = ['date','team','formation'])
formations_df['formation'] = formations_df['formation'].astype('object')
i = 0


#POSSIBILE FIX: TENGO UN DIZIONARIO CON TEAM COME CHIAVE E HAS_PLAYED COME VALORE: METTO HAS_PLAYED A 1 QUANDO UN TEAM GIOCA, E LO RIMETTO A ZERO QUANDO SCARICO LE STATISTICHE. SE HAS_PLAYED è A 0
#SIGNIFICA CHE NON DEVO SCARICARE NUOVE STATISTICHE 


reset_dict(info_dict)

first_match_dates = [pd.to_datetime(x).date() for x in league_df.date.unique()[:3]]  #date dei primi 3 giorni di partita (non posso usare matchday perchè ci sono i posticipi)

for x,date in enumerate(tqdm(dates)):

  #if(x == 6):
    #break


  #print("data corrente: ", date)
  previous_day = get_date_range(date,1)

  
  for team in teams:   


    names = list(team_names[team_names.squadra == team].soprannome)
    team = remove_prefix(team).lower()

    #print("team corrente: ", team)

    date_from = get_date_range(date,3)
    query = league_df[((league_df.team == team) & (league_df.date == date))]
 
    if len(query) == 0: #se questo team in questa data non ha giocato skippo
      #print("il team non ha giocato, skippo")
      continue
    #else:
    #  print(f"trovata partita: {team}-{query.opponent.values[0]}")


    #print(info_dict)
    if info_dict[team] == 1:  #se trovo un match in cui il team ha giocato (precedentemente) scarico le nuove stats, altrimenti tengo quelle già scaricate
      #print("uno dei due team ha giocato precedentemente e ha bigogno di nuove statistiche, le scarico")
      leaderboard,players_stats = get_tables('', previous_day.strftime("%m-%d-%Y"),d)
      reset_dict(info_dict) #resetto per tutti i team dato che la get_tables scarica per tutti


    #print("il ", team, "ha giocato, me lo segno")
    info_dict[team] = 1 #metto che il team ha giocato


    if date in first_match_dates: #skippo le prime partite perchè non posso calcolare le feature aggregate precedenti ad esse
      #print("è la prima giormata quindi non dispongo di nessuna statistica precedente, skippo")
      continue

    #print(team)

    league_df.loc[query.index, 'rank_team'] = int(leaderboard[leaderboard.Team.apply(remove_prefix).apply(str.lower) == query.iloc[0]['team']].head(1).index.values)
    league_df.loc[query.index, 'rank_opponent'] = int(leaderboard[leaderboard.Team.apply(remove_prefix).apply(str.lower) == query.iloc[0]['opponent']].head(1).index.values)
    tmp = sentences_ita[[((('probabile formazione:' in sentence) or ('probabile formazione :' in sentence)) and (((team + " ") in sentence) or any([((name + " ") in sentence) for name in names]))) for sentence in sentences_ita.text] & (sentences_ita.date.apply(str_to_date) >= date_from) & (sentences_ita.date.apply(str_to_date) <= date)]
    
    if len(tmp) == 1:
      players = re.split('formazione: |formazione :',list(tmp['text'])[0])[1].replace(",",";").replace(".","").split(";")
      formations_df.loc[i,'date'] = date
      formations_df.loc[i,'team'] = team
      formations_df.at[i,'formation'] = [player[1:] for player in players]
      i+=1
    elif len(tmp) == 0:
      formations_df.loc[i,'date'] = date
      formations_df.loc[i,'team'] = team
      formations_df.at[i,'formation'] = None
      i+=1
    else:
      #prendo la prima riga che è la più recente
      recent_formation = tmp.iloc[:1]
      players = re.split('formazione: |formazione :',list(recent_formation['text'])[0])[1].replace(",",";").replace(".","").split(";")
      formations_df.loc[i,'date'] = date
      formations_df.loc[i,'team'] = team
      formations_df.at[i,'formation'] = [player[:1] for player in players]
      i+=1

    lineups = matches_df[(matches_df.hteam_name == team) & (matches_df.date.apply(lambda x : datetime.datetime.strptime(x,"%Y-%m-%dT%H:%M:%SZ").date()) == date)][['hteam_lineup','ateam_lineup']]

    h_lineup = lineups['hteam_lineup']
    a_lineup = lineups['ateam_lineup'] 

    hteam_players = []  #non reinizializzando rimane quello vecchio che contiene roba e quindi l'if len(hteam_players) > 0 è sempre vero dopo il primo
    ateam_players = []  

    

    if len(h_lineup) > 0:
      hteam_players = players_stats[players_stats.Player.apply(lambda x : any(lev.ratio(x.lower(),name)>=0.9 for name in list(h_lineup)[0]))]
      
    if len(a_lineup) > 0:
      ateam_players = players_stats[players_stats.Player.apply(lambda x : any(lev.ratio(x.lower(),name)>=0.9 for name in list(a_lineup)[0]))]



    if len(hteam_players) > 0: #and len(hteam_players) == 11:
      matches_df.at[h_lineup.index,'hteam_xGoals'] = hteam_players.groupby('Player').apply(lambda x : (x['G'] - x['xG'])).sum()
      matches_df.at[h_lineup.index,'hteam_xAssists'] = hteam_players.groupby('Player').apply(lambda x : (x['A'] - x['xA'])).sum()

    if len(ateam_players) > 0: #and len(hteam_players) == 11:
      matches_df.at[a_lineup.index,'ateam_xGoals'] = ateam_players.groupby('Player').apply(lambda x : (x['G'] - x['xG'])).sum()
      matches_df.at[a_lineup.index,'ateam_xAssists'] = ateam_players.groupby('Player').apply(lambda x : (x['A'] - x['xA'])).sum()

    lineup = formations_df[(formations_df.team == team) & (formations_df.date == date)].formation  #cagliari | None
    if len(lineup) > 0:
      try:
        team_players = players_stats[players_stats.Player.apply(lambda x : any(lev.ratio((x.split(" ")[1].lower() if len(x.split(" ")) >1 else x.lower()) ,(name.split(" ")[1].lower() if len(name.split(" ")) >1 else name))>=0.9 for name in list(lineup)[0]))]
        if len(team_players) > 0:
          formations_df.at[lineup.index, 'team_xGoals'] = team_players.groupby('Player').apply(lambda x : (x['G'] - x['xG'])).sum()
          formations_df.at[lineup.index, 'team_xAssists'] = team_players.groupby('Player').apply(lambda x : (x['A'] - x['xA'])).sum()
      except TypeError: #team_player == None
        previous_lineup_home = matches_df[(matches_df.hteam_name == team) & (matches_df.date.apply(lambda x : datetime.datetime.strptime(x,"%Y-%m-%dT%H:%M:%SZ").date()) < date)][['date','hteam_lineup']].tail(1)
        #print(previous_lineup_home)
        previous_lineup_away = matches_df[(matches_df.ateam_name == team) & (matches_df.date.apply(lambda x : datetime.datetime.strptime(x,"%Y-%m-%dT%H:%M:%SZ").date()) < date)][['date','ateam_lineup']].tail(1)
        #print(previous_lineup_away)
        if len(previous_lineup_home) > 0 and len(previous_lineup_away) > 0:
          previous_lineup = (previous_lineup_home if previous_lineup_home.values[0][0] > previous_lineup_away.values[0][0] else previous_lineup_away)
        
        if len(previous_lineup_home) > 0 and not(len(previous_lineup_away) > 0):
          previous_lineup = previous_lineup_home
        if len(previous_lineup_away) > 0 and not(len(previous_lineup_home) > 0):
          previous_lineup = previous_lineup_away

        if len(previous_lineup_home) == 0 and len(previous_lineup_away) == 0:
            formations_df.at[lineup.index, 'team_xGoals'] = None
            formations_df.at[lineup.index, 'team_xAssists'] = None

        else:                                               
          team_players = players_stats[players_stats.Player.apply(lambda x : any(lev.ratio(x.lower() ,name)>=0.9 for name in list(previous_lineup.values)[0][1]))]
          if len(team_players) > 0:
            formations_df.at[lineup.index, 'team_xGoals'] = team_players.groupby('Player').apply(lambda x : (x['G'] - x['xG'])).sum()
            formations_df.at[lineup.index, 'team_xAssists'] = team_players.groupby('Player').apply(lambda x : (x['A'] - x['xA'])).sum()

        





  0%|          | 0/111 [00:00<?, ?it/s]

In [33]:
formations_df

,date,team,formation,team_xGoals,team_xAssists
0,2020-09-26,fiorentina,"[dragowski, milenkovic, pezzella, caceres, chi...",-1.13,-0.56
1,2020-09-26,atalanta,"[ello, toloi, caldara, sutalo, hateboer, de ro...",NaN,NaN
2,2020-09-26,cagliari,None,0.04,0.15
3,2020-09-26,inter,"[handanovic, d’ambrosio, bastoni, kolarov, hak...",NaN,NaN
4,2020-09-26,lazio,None,NaN,NaN
...,...,...,...,...,...
741,2021-05-23,verona,"[pandur, dawidowicz, gunter, dimarco, faraoni,...",-0.64,-5.26
742,2021-05-23,sassuolo,None,3.51,0.15
743,2021-05-23,spezia,None,-0.02,0.29
744,2021-05-23,torino,None,1.39,-0.30


In [ ]:
from thinc.layers.list2array import list2array
formations_df = pd.DataFrame(columns = ['date','team','formation'])
formations_df['formation'] = formations_df['formation'].astype('object')
i = 0


#ADESSO STIAMO PRENDENDO LE 3 DATE PRECEDENTI ALLE 3 DATE DI UNA "GIORNATA", INVECE DOBBIAMO PRENDERE SOLO QUELLA PRECEDENTE ALLA PRIMA DATA DELLA GIORNATA, PER OGNI GIORNATA, E ITERARE SOLO SU QUESTE DATE


first_match_dates = [pd.to_datetime(x).date() for x in league_df.date.unique()[:3]]  #date dei primi 3 giorni di partita (non posso usare matchday perchè ci sono i posticipi)

for x,date in enumerate(tqdm(dates)):

  #if(x == 6):
    #break

  if date in first_match_dates: #skippo le prime partite perchè non posso calcolare le feature aggregate precedenti ad esse
    continue

  print(date)
  previous_day = get_date_range(date,1)
  leaderboard,players_stats = get_tables('', previous_day.strftime("%m-%d-%Y"),d) 

  
  for team in teams:   
    names = list(team_names[team_names.squadra == team].soprannome)
    team = remove_prefix(team).lower()

    date_from = get_date_range(date,3)
    query = league_df[((league_df .team == team) & (league_df.date == date))]
 
    if len(query) == 0:
      continue

    #print(team)

    league_df.loc[query.index, 'rank_team'] = int(leaderboard[leaderboard.Team.apply(remove_prefix).apply(str.lower) == query.iloc[0]['team']].head(1).index.values)
    league_df.loc[query.index, 'rank_opponent'] = int(leaderboard[leaderboard.Team.apply(remove_prefix).apply(str.lower) == query.iloc[0]['opponent']].head(1).index.values)
    tmp = sentences_ita[[((('probabile formazione:' in sentence) or ('probabile formazione :' in sentence)) and (((team + " ") in sentence) or any([((name + " ") in sentence) for name in names]))) for sentence in sentences_ita.text] & (sentences_ita.date.apply(str_to_date) >= date_from) & (sentences_ita.date.apply(str_to_date) <= date)]
    
    if len(tmp) == 1:
      players = re.split('formazione: |formazione :',list(tmp['text'])[0])[1].replace(",",";").replace(".","").split(";")
      formations_df.loc[i,'date'] = date
      formations_df.loc[i,'team'] = team
      formations_df.at[i,'formation'] = [player[1:] for player in players]
      i+=1
    elif len(tmp) == 0:
      formations_df.loc[i,'date'] = date
      formations_df.loc[i,'team'] = team
      formations_df.at[i,'formation'] = None
      i+=1
    else:
      #prendo la prima riga che è la più recente
      recent_formation = tmp.iloc[:1]
      players = re.split('formazione: |formazione :',list(recent_formation['text'])[0])[1].replace(",",";").replace(".","").split(";")
      formations_df.loc[i,'date'] = date
      formations_df.loc[i,'team'] = team
      formations_df.at[i,'formation'] = [player[:1] for player in players]
      i+=1

    lineups = matches_df[(matches_df.hteam_name == team) & (matches_df.date.apply(lambda x : datetime.datetime.strptime(x,"%Y-%m-%dT%H:%M:%SZ").date()) == date)][['hteam_lineup','ateam_lineup']]

    h_lineup = lineups['hteam_lineup']
    a_lineup = lineups['ateam_lineup'] 

    hteam_players = []  #non reinizializzando rimane quello vecchio che contiene roba e quindi l'if len(hteam_players) > 0 è sempre vero dopo il primo
    ateam_players = []  

    

    if len(h_lineup) > 0:
      hteam_players = players_stats[players_stats.Player.apply(lambda x : any(lev.ratio(x.lower(),name)>=0.9 for name in list(h_lineup)[0]))]
      
    if len(a_lineup) > 0:
      ateam_players = players_stats[players_stats.Player.apply(lambda x : any(lev.ratio(x.lower(),name)>=0.9 for name in list(a_lineup)[0]))]



    if len(hteam_players) > 0: #and len(hteam_players) == 11:
      matches_df.at[h_lineup.index,'hteam_xGoals'] = hteam_players.groupby('Player').apply(lambda x : (x['G'] - x['xG'])).sum()
      matches_df.at[h_lineup.index,'hteam_xAssists'] = hteam_players.groupby('Player').apply(lambda x : (x['A'] - x['xA'])).sum()

    if len(ateam_players) > 0: #and len(hteam_players) == 11:
      matches_df.at[a_lineup.index,'ateam_xGoals'] = ateam_players.groupby('Player').apply(lambda x : (x['G'] - x['xG'])).sum()
      matches_df.at[a_lineup.index,'ateam_xAssists'] = ateam_players.groupby('Player').apply(lambda x : (x['A'] - x['xA'])).sum()

    lineup = formations_df[(formations_df.team == team) & (formations_df.date == date)].formation  #cagliari | None
    if len(lineup) > 0:
      try:
        team_players = players_stats[players_stats.Player.apply(lambda x : any(lev.ratio((x.split(" ")[1].lower() if len(x.split(" ")) >1 else x.lower()) ,(name.split(" ")[1].lower() if len(name.split(" ")) >1 else name))>=0.9 for name in list(lineup)[0]))]
        if len(team_players) > 0:
          formations_df.at[lineup.index, 'team_xGoals'] = team_players.groupby('Player').apply(lambda x : (x['G'] - x['xG'])).sum()
          formations_df.at[lineup.index, 'team_xAssists'] = team_players.groupby('Player').apply(lambda x : (x['A'] - x['xA'])).sum()
      except TypeError: #team_player == None
        previous_lineup_home = matches_df[(matches_df.hteam_name == team) & (matches_df.date.apply(lambda x : datetime.datetime.strptime(x,"%Y-%m-%dT%H:%M:%SZ").date()) < date)][['date','hteam_lineup']].tail(1)
        #print(previous_lineup_home)
        previous_lineup_away = matches_df[(matches_df.ateam_name == team) & (matches_df.date.apply(lambda x : datetime.datetime.strptime(x,"%Y-%m-%dT%H:%M:%SZ").date()) < date)][['date','ateam_lineup']].tail(1)
        #print(previous_lineup_away)
        if len(previous_lineup_home) > 0 and len(previous_lineup_away) > 0:
          previous_lineup = (previous_lineup_home if previous_lineup_home.values[0][0] > previous_lineup_away.values[0][0] else previous_lineup_away)
        
        if len(previous_lineup_home) > 0 and not(len(previous_lineup_away) > 0):
          previous_lineup = previous_lineup_home
        if len(previous_lineup_away) > 0 and not(len(previous_lineup_home) > 0):
          previous_lineup = previous_lineup_away

        if len(previous_lineup_home) == 0 and len(previous_lineup_away) == 0:
            formations_df.at[lineup.index, 'team_xGoals'] = None
            formations_df.at[lineup.index, 'team_xAssists'] = None

        else:                                               
          team_players = players_stats[players_stats.Player.apply(lambda x : any(lev.ratio(x.lower() ,name)>=0.9 for name in list(previous_lineup.values)[0][1]))]
          if len(team_players) > 0:
            formations_df.at[lineup.index, 'team_xGoals'] = team_players.groupby('Player').apply(lambda x : (x['G'] - x['xG'])).sum()
            formations_df.at[lineup.index, 'team_xAssists'] = team_players.groupby('Player').apply(lambda x : (x['A'] - x['xA'])).sum()

        


#quando l'hai fatto?





In [36]:
#Iterate through dictionaries of statistics

for i,match in enumerate(matches_df.values):
  stats = matches_df.loc[i,'statistics']
  #print(len(stats)*3)
  #print(stats)
  y = 0
  for stat in stats:
    
    matches_df.at[i,f'{stat["type"].replace(" ", "_")}_home'] = stat['home']
    matches_df.at[i,f'{stat["type"].replace(" ", "_")}_away'] = stat['away']
    y += 2

  #print(y)

In [44]:
formations_df.groupby('team').get_group('milan')

,date,team,formation,team_xGoals,team_xAssists
8,2020-09-27,milan,"[donnarumma, calabria, kjaer, gabbia, theo her...",-0.81,-0.79
32,2020-10-04,milan,"[g donnarumma, calabria, gabbia, kjaer, theo h...",-1.69,-0.66
42,2020-10-17,milan,"[g donnarumma, calabria, kjaer, romagnoli, the...",-0.70,-0.91
80,2020-10-26,milan,"[atarusanu, calabria, kjaer, romagnoli, theo h...",-0.66,-1.24
88,2020-11-01,milan,"[donnarumma, calabria, kjaer, romagnoli, theo ...",0.20,-2.45
112,2020-11-08,milan,None,3.02,1.38
130,2020-11-22,milan,"[donnarumma, calabria, kjaer, romagnoli, herna...",-0.88,-3.74
150,2020-11-29,milan,"[donnarumma, calabria, kjaer, romagnoli, theo ...",-2.24,-3.42
170,2020-12-06,milan,"[donnarumma, calabria, gabbia, romagnoli, theo...",-2.36,-4.55
190,2020-12-13,milan,"[donnarumma, calabria, gabbia, romagnoli, theo...",-4.01,-2.15


In [34]:
matches_df[['date','hteam_name','ateam_name','hteam_score','ateam_score','hteam_xGoals','hteam_xAssists','ateam_xGoals','ateam_xAssists']] 

,date,hteam_name,ateam_name,hteam_score,ateam_score,hteam_xGoals,hteam_xAssists,ateam_xGoals,ateam_xAssists
0,2020-09-19T18:00:00Z,fiorentina,torino,1,0,NaN,NaN,NaN,NaN
1,2020-09-19T20:45:00Z,verona,roma,3,0,NaN,NaN,NaN,NaN
2,2020-09-20T12:30:00Z,parma,napoli,0,2,NaN,NaN,NaN,NaN
3,2020-09-20T15:00:00Z,genoa,crotone,4,1,NaN,NaN,NaN,NaN
4,2020-09-20T18:00:00Z,sassuolo,cagliari,1,1,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
375,2021-05-23T20:45:00Z,sassuolo,lazio,2,0,1.30,1.80,-3.20,3.08
376,2021-05-23T20:45:00Z,atalanta,milan,0,2,0.82,5.52,-0.45,4.28
377,2021-05-23T20:45:00Z,spezia,roma,2,2,1.40,5.60,2.40,3.04
378,2021-05-23T15:00:00Z,inter,udinese,5,1,-0.30,4.17,-0.73,-1.08


In [40]:
#hteam_xGoals hteam_xAssists ateam_xGoals ateam_xAssists Shots On Goal_home Shots On Goal_away Ball Possession_home Ball Possession_away Passes Accurate_home Passes Accurate_away

#index = 0

for index,match in enumerate(tqdm(league_df.values)):
  if index%2==0:
    #q = matches_df[matches_df.hteam_name == match[4] & matches_df.date == match[1]]
    q = matches_df[((matches_df.hteam_name == match[4])) & (matches_df.date.apply(lambda x : datetime.datetime.strptime(x,"%Y-%m-%dT%H:%M:%SZ").date()) == match[1])]
    #print(len(q))
    #if len(q) > 0:
    league_df.loc[index,'team_xGoals']  =  q.iloc[0]['hteam_xGoals']
    league_df.loc[index,'team_xAssists'] = q.iloc[0]['hteam_xAssists']
    league_df.loc[index,'opp_xGoals'] = q.iloc[0]['ateam_xGoals']
    league_df.loc[index,'opp_xAssists'] = q.iloc[0]['ateam_xAssists']
    #league_df['team_shots_total'] = league_df['Shots_Total_home'].astype('int')
    #league_df['opp_shots_total'] = league_df['Shots_Total_away'].astype('int')
    league_df.loc[index,'team_shots_total'] = int(q.iloc[0]['Shots_Total_home']) #provo con total adesso poi vediamo
    league_df.loc[index,'opp_shots_total'] = int(q.iloc[0]['Shots_Total_away'])
    #league_df['team_ball_possession'] = league_df['team_ball_possession'].astype('float')
    #league_df['opp_ball_possession'] = league_df['opp_ball_possession'].astype('float')
    league_df.loc[index,'team_ball_possession'] = int(q.iloc[0]['Ball_Possession_home'].replace('%',''))/100
    league_df.loc[index,'opp_ball_possession'] = int(q.iloc[0]['Ball_Possession_away'].replace('%',''))/100
    #passes_accurate non mi convince adesso. Vediamo poi cosa si può aggiungere o anche combinare più statistiche in una.
    if index <= len(league_df)-2: 

      index+=1
      league_df.loc[index,'team_xGoals']  =  q.iloc[0]['ateam_xGoals']
      league_df.loc[index,'team_xAssists'] = q.iloc[0]['ateam_xAssists']
      league_df.loc[index,'opp_xGoals'] = q.iloc[0]['hteam_xGoals']
      league_df.loc[index,'opp_xAssists'] = q.iloc[0]['hteam_xAssists']
      league_df.loc[index,'team_shots_total'] = q.iloc[0]['Shots_Total_away'] #provo con total adesso poi vediamo
      league_df.loc[index,'opp_shots_total'] = q.iloc[0]['Shots_Total_home']
      league_df.loc[index,'team_ball_possession'] = int(q.iloc[0]['Ball_Possession_away'].replace('%',''))/100
      league_df.loc[index,'opp_ball_possession'] = int(q.iloc[0]['Ball_Possession_home'].replace('%',''))/100

  #index+=1


  0%|          | 0/760 [00:00<?, ?it/s]

In [41]:
league_df.loc[35:45]

,match_id,date,time,matchday,team,opponent,place,result,s_goals,c_goals,...,rank_team,rank_opponent,team_xGoals,team_xAssists,opp_xGoals,opp_xAssists,team_shots_total,opp_shots_total,team_ball_possession,opp_ball_possession
35,59176,2020-09-30,18,1,inter,benevento,A,W,5,2,...,9.0,10.0,0.05,0.98,0.70,0.09,18,11,0.67,0.33
36,59177,2020-09-30,18,1,udinese,spezia,H,L,0,2,...,15.0,17.0,-2.15,-0.98,-0.27,0.40,13,4,0.48,0.52
37,59177,2020-09-30,18,1,spezia,udinese,A,W,2,0,...,17.0,15.0,-0.27,0.40,-2.15,-0.98,4,13,0.52,0.48
38,59178,2020-09-30,20,1,lazio,atalanta,H,L,1,4,...,7.0,6.0,0.64,-0.54,-0.18,0.76,14,10,0.60,0.40
39,59178,2020-09-30,20,1,atalanta,lazio,A,W,4,1,...,6.0,7.0,-0.18,0.76,0.64,-0.54,10,14,0.40,0.60
40,59189,2020-10-02,20,3,fiorentina,sampdoria,H,L,1,2,...,11.0,18.0,-0.03,-0.84,-0.03,0.16,22,18,0.64,0.36
41,59189,2020-10-02,20,3,sampdoria,fiorentina,A,W,2,1,...,18.0,11.0,-0.03,0.16,-0.03,-0.84,18,22,0.36,0.64
42,59190,2020-10-03,15,3,sassuolo,crotone,H,W,4,1,...,6.0,19.0,-0.03,1.70,-0.74,-0.21,7,15,0.54,0.46
43,59190,2020-10-03,15,3,crotone,sassuolo,A,L,1,4,...,19.0,6.0,-0.74,-0.21,-0.03,1.70,15,7,0.46,0.54
44,59191,2020-10-03,20,3,udinese,roma,H,L,0,1,...,18.0,15.0,-3.19,-1.22,-2.55,-2.12,10,13,0.44,0.56


In [46]:
league_df

,match_id,date,time,matchday,team,opponent,place,result,s_goals,c_goals,...,rolling_s_goals,rolling_c_goals,rolling_team_xGoals,rolling_team_xAssists,rolling_opp_xGoals,rolling_opp_xAssists,rolling_team_shots_total,rolling_opp_shots_total,rolling_team_ball_possession,rolling_opp_ball_possession
0,59169,2020-09-19,18,1,fiorentina,torino,H,W,1,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,59169,2020-09-19,18,1,torino,fiorentina,A,L,0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,59170,2020-09-19,20,1,verona,roma,H,W,3,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,59170,2020-09-19,20,1,roma,verona,A,L,0,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,59171,2020-09-20,12,1,parma,napoli,H,L,0,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
755,59518,2021-05-23,20,38,roma,spezia,A,D,2,2,...,2.0,1.6,-4.218,-0.824,2.490,1.224,16.4,10.8,0.554,0.446
756,59519,2021-05-23,15,38,inter,udinese,H,W,5,1,...,2.6,1.0,3.928,3.590,-0.934,-0.426,14.2,13.0,0.552,0.448
757,59519,2021-05-23,15,38,udinese,inter,A,L,1,5,...,1.4,2.2,-0.036,-0.798,-0.370,0.412,7.8,17.8,0.464,0.536
758,59520,2021-05-23,20,38,napoli,verona,H,D,1,1,...,2.8,0.6,5.902,2.208,-1.216,-0.298,19.8,9.6,0.542,0.458


In [45]:
#dividere il dataframe per ogni team per calcolare le medie
#dobbiamo calcolare la media di tutti gli expected goal e assist ,tiri in porta, possesso palla, passaggi precisi   

new_col_l = pd.Series([],dtype=float)
new_col_f = pd.Series([],dtype=float)

grp_l = league_df.groupby('team')

grp_f = formations_df.groupby('team')

cols = ['s_goals','c_goals','team_xGoals','team_xAssists',	'opp_xGoals',	'opp_xAssists',	'team_shots_total',	'opp_shots_total','team_ball_possession',	'opp_ball_possession']

for team in teams:

  tmp_l = grp_l.get_group(remove_prefix(team).lower()).sort_values('date')[cols].rolling(window=5, min_periods=0, closed ='left').mean() #aggiungo colonna rolling_s_goals in fondo?
  
  tmp_f = grp_f.get_group(remove_prefix(team).lower()).sort_values('date')[['team_xGoals','team_xAssists']].rolling(window=5, min_periods=0, closed ='left').mean()

  
  new_col_l = pd.concat([new_col_l,tmp_l])
  new_col_f = pd.concat([new_col_f,tmp_f])

league_df[[f'rolling_{col}' for col in cols]] = new_col_l[cols]
formations_df['rolling_team_xGoals'] = new_col_f['team_xGoals']
formations_df['rolling_team_xAssists'] = new_col_f['team_xAssists']




#creare due dataframe (train e test) con tutte le features  (con tiri in porta, possesso palla, passaggi precisi)


In [52]:
formations_df

,date,team,formation,team_xGoals,team_xAssists,rolling_team_xGoals,rolling_team_xAssists
0,2020-09-26,fiorentina,"[dragowski, milenkovic, pezzella, caceres, chi...",-1.13,-0.56,NaN,NaN
1,2020-09-26,atalanta,"[ello, toloi, caldara, sutalo, hateboer, de ro...",NaN,NaN,NaN,NaN
2,2020-09-26,cagliari,None,0.04,0.15,NaN,NaN
3,2020-09-26,inter,"[handanovic, d’ambrosio, bastoni, kolarov, hak...",NaN,NaN,NaN,NaN
4,2020-09-26,lazio,None,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
741,2021-05-23,verona,"[pandur, dawidowicz, gunter, dimarco, faraoni,...",-0.64,-5.26,-5.696000,-5.4920
742,2021-05-23,sassuolo,None,3.51,0.15,0.735000,-2.4375
743,2021-05-23,spezia,None,-0.02,0.29,0.002500,2.6725
744,2021-05-23,torino,None,1.39,-0.30,2.166667,-1.2500


In [53]:
#formations_df.to_csv("./formations.csv")
#league_df.to_csv("./league.csv")
#matches_df.to_csv("./matches.csv")

In [ ]:
matches_df[(matches_df.date.apply(lambda x : datetime.datetime.strptime(x,"%Y-%m-%dT%H:%M:%SZ").date()) == league_df['date'][0])]

,match_id,date,day,hteam_name,ateam_name,hteam_score,ateam_score,winner,hteam_lineup,ateam_lineup,...,Passes_Total_home,Passes_Total_away,Passes_Accurate_home,Passes_Accurate_away,Red_Cards_home,Red_Cards_away,hteam_xGoals,hteam_xAssists,ateam_xGoals,ateam_xAssists
0,59169,2020-09-19T18:00:00Z,1,fiorentina,torino,1,0,H,"[federico ceccherini, cristiano biraghi, martí...","[cristian ansaldi, soualiho meïté, armando izz...",...,423,445,361,369,NaN,NaN,NaN,NaN,NaN,NaN
1,59170,2020-09-19T20:45:00Z,1,verona,roma,3,0,H,"[koray günter, adrien tameze, miguel veloso, s...","[antonio mirante, leonardo spinazzola, jordan ...",...,351,520,237,400,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
import pandas as pd

df = pd.DataFrame({'B': [0, 1, 2, 100, 4]})
df
 


In [ ]:
df['rolling']= df['B'].rolling(window=4,min_periods=0, closed ='left').mean()

In [ ]:
df

,B,rolling
0,0,NaN
1,1,0.00
2,2,0.50
3,100,1.00
4,4,25.75
